# STA 141B Lecture 14

February 28, 2023


### Topics

* Databases & SQL

### Datasets

* The Suppliers Database

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](https://www.sqltutorial.org/sql-cheat-sheet/)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

In [ ]:
import numpy as np
import pandas as pd

import sqlite3 as sql

In [ ]:
db = sql.connect("data/suppliers.sqlite")

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master;", db)

In [ ]:
pd.read_sql("SELECT * FROM parts;", db)

### `WHERE`

`WHERE` puts conditions on the rows returned. `WHERE` is the SQL equivalent of subsetting.

You can use `=` to test equality. Other comparison operators, such as `>=`, are also available.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE weight = 17", db)

You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE city = 'London' OR color = 'Red';", db)

You can use `IN` to check whether a value is in a collection of values.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE city IN ('Paris', 'London');", db)

SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex `.*`
* `_` matches any one character, similar to regex `.`

In other databases (but not SQLite):
* `[]` matches any one of the characters you put inside the brackects, identical to regex `[]`

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE city LIKE '%s';", db)

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE city LIKE 'Pari_';", db)

The `BETWEEN` keyword is useful for selecting ranges.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE weight BETWEEN 14 AND 20;", db)

### Operators

You can use arithmetic operators `+`, `-`, `*`, `\`, `%` on SQL columns to perform columnwise computations. These are the SQL equivalent of vectorized arithmetic.

In [ ]:
pd.read_sql("SELECT weight * weight AS squared_weight, * FROM parts;", db)

In [ ]:
pd.read_sql("SELECT weight * weight AS squared_weight, * FROM parts WHERE squared_weight > 300;", db)

### `AS`

You can rename a column with the `AS` keyword. This keyword is especially useful together with SQL arithmetic operators and functions.

### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM parts;", db)

In [ ]:
pd.read_sql("SELECT UPPER(City), * FROM parts LIMIT 3;", db)

### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [ ]:
pd.read_sql("SELECT AVG(weight) FROM parts;", db)

In [ ]:
pd.read_sql("SELECT AVG(weight), city FROM parts GROUP BY city;", db)

In [ ]:
pd.read_sql("SELECT AVG(weight), city, partname FROM parts GROUP BY city, partname;", db)

### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [ ]:
# It's an error to use WHERE to put a condition on an aggregated column.
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city WHERE avg_weight > 15", db)

In [ ]:
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city HAVING avg_weight > 15", db)

### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts", db)

In [ ]:
pd.read_sql("""
    SELECT * FROM
    supplierparts
    LEFT JOIN
    parts
    ON supplierparts.partid = parts.partid
""", db)

SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

In [ ]:
pd.read_sql("SELECT * FROM suppliers", db)

In [ ]:
pd.read_sql("""
    SELECT * FROM
    supplierparts AS l
    INNER JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

In [ ]:
pd.read_sql("""
    SELECT l.qty, r.* FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

### Subqueries

You can write one or more `SELECT` queries within another:

In [ ]:
pd.read_sql("""
    SELECT * FROM
        (SELECT l.*, r.supplierid, r.qty FROM
        parts AS l
        INNER JOIN
        supplierparts AS r
        ON l.partid = r.partid) AS a
    INNER JOIN
        (SELECT * FROM suppliers
        WHERE status > 10) AS b
    ON a.supplierid = b.supplierid;
""", db)

In [ ]:
# You can paste strings into SQL queries, but make sure the string you paste in isn't
# compromised (e.g., user input)
val = pd.read_sql("SELECT AVG(weight) FROM parts", db).iloc[0, 0]
query = "SELECT * FROM parts WHERE weight > {}".format(val)
pd.read_sql(query, db)

In [ ]:
pd.read_sql("""
    SELECT * FROM parts WHERE weight > (
        SELECT AVG(weight) FROM parts
    )
""", db)

### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

In [ ]:
result = db.execute("""
    CREATE TEMPORARY TABLE table2 AS
        SELECT l.*, r.qty, r.supplierid FROM
            parts AS l
        INNER JOIN
            supplierparts AS r
        ON l.partid = r.partid
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM table2", db)

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db)

In [ ]:
db.execute("DROP TABLE my_table;")

db.commit()

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///data/suppliers.sqlite")
conn = engine.raw_connection()

In [ ]:
conn

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

In [ ]:
engine.table_names()

In [ ]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.